In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
!pip install torch

In [ ]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

In [ ]:
import torchvision

In [ ]:
from PIL import Image
import shutil
import os
import numpy as np
import argparse

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install split_folders

ModuleNotFoundError: ignored

In [ ]:
import splitfolders

In [ ]:
input_folder = "/content/drive/MyDrive/qmul_toplogo10/jpg"
output = "/content/drive/MyDrive/qmul_toplogo10" #where you want the split datasets saved. one will be created if none is set

splitfolders.ratio(input_folder, output=output, seed=42, ratio=(.8, .1, .1)) # ratio of split are in order of train/val/test. You can change to whatever you want. For train/val sets only, you could do .75, .25 for example.

Copying files: 700 files [00:09, 72.13 files/s]


In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])


In [ ]:
input_path = '/content/drive/MyDrive/qmul_toplogo10/'

In [ ]:
data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'train': 
    datasets.ImageFolder(input_path + 'train', data_transforms['train']),
    'validation': 
    datasets.ImageFolder(input_path + 'validation', data_transforms['validation'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=32,
                                shuffle=True,
                                num_workers=0),  # for Kaggle
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=32,
                                shuffle=False,
                                num_workers=0)  # for Kaggle
}

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cpu')

In [ ]:
model = models.resnet50(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = False   
    
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               #nn.Softmax(),
               nn.Linear(128, 10)).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model

In [ ]:
model_trained = train_model(model, criterion, optimizer, num_epochs=50)

Epoch 1/50
----------
train loss: 2.2723, acc: 0.1536
validation loss: 2.0975, acc: 0.2143
Epoch 2/50
----------
train loss: 2.0271, acc: 0.3250
validation loss: 2.0111, acc: 0.2571
Epoch 3/50
----------
train loss: 1.8415, acc: 0.3607
validation loss: 1.7672, acc: 0.4143
Epoch 4/50
----------
train loss: 1.6441, acc: 0.4946
validation loss: 1.7001, acc: 0.3571
Epoch 5/50
----------
train loss: 1.5500, acc: 0.5054
validation loss: 1.6614, acc: 0.3286
Epoch 6/50
----------
train loss: 1.4597, acc: 0.5411
validation loss: 1.5726, acc: 0.3714
Epoch 7/50
----------
train loss: 1.3288, acc: 0.5625
validation loss: 1.4880, acc: 0.4714
Epoch 8/50
----------
train loss: 1.2371, acc: 0.6125
validation loss: 1.4900, acc: 0.5000
Epoch 9/50
----------
train loss: 1.1915, acc: 0.6214
validation loss: 1.4590, acc: 0.4571
Epoch 10/50
----------
train loss: 1.1766, acc: 0.6250
validation loss: 1.4843, acc: 0.4429
Epoch 11/50
----------
train loss: 1.1563, acc: 0.6214
validation loss: 1.5029, acc: 0.42